# Unzip The Augmented Data Folder

In [38]:
!unzip "/content/data.zip"

Archive:  /content/data.zip
replace data/Testing/Corpse_AUG/aug_download (1)_0_2820.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: all
error:  invalid response [all]
replace data/Testing/Corpse_AUG/aug_download (1)_0_2820.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: all
error:  invalid response [all]
replace data/Testing/Corpse_AUG/aug_download (1)_0_2820.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: none
replace data/Testing/Corpse_AUG/aug_download (1)_0_2845.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: none
replace data/Testing/Corpse_AUG/aug_download (1)_0_2922.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: r
new name: s
  inflating: s                       
replace data/Testing/Corpse_AUG/aug_download (1)_0_3924.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/Testing/Corpse_AUG/aug_download (1)_0_3924.jpg  
  inflating: data/Testing/Corpse_AUG/aug_download (1)_0_4240.jpg  
  inflating: data/Testing/Corpse_AUG/aug_download (1)_0_8116.jpg  
  inflating: data/Testing/Corpse_AUG/aug_do

# Import The Required Libraries

In [39]:
import os, gc, glob, random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras
from PIL import Image
from keras.layers import Dense,Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

# Load The Train And Test Datasets

In [40]:
#initializing the datagenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator()

#loading the train dataset from the directory
traindata= datagen.flow_from_directory(directory="/content/data/Training",target_size=(224,224),batch_size=100)

#loading the test dataset from the directory
testdata= datagen.flow_from_directory(directory="/content/data/Testing",target_size=(224,224),batch_size=100)

Found 2916 images belonging to 6 classes.
Found 1759 images belonging to 6 classes.


# Model Building Using CNN

In [41]:
base_model = InceptionV3(input_shape=(224,224,3),include_top=False)

#Freezing the weights in the pretrained model

In [42]:
for layer in base_model.layers:
  layer.trainable = False

#Flatening and adding dense layer


In [43]:
X = Flatten()(base_model.output)

#using sigmoid function as activation funciton
X = Dense(units = 6, activation = 'sigmoid')(X)

# Compiling the model

In [44]:
model = Model(base_model.input,X)

model.compile(optimizer = 'adam',loss = keras.losses.binary_crossentropy,metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 111, 111, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                           

# Model Checkpoint and Early Stopping

In [45]:
# Creating a model checkpoint which monitors the accuracy of the model and saves the checkpoint
mc = ModelCheckpoint(filepath = "./DN.h5",
                     monitor = 'accuracy',
                     verbose = 1,
                     save_best_only = True)

# Creating a earlystopping object which stop training once the model performance stops improving on a hold out validation dataset
es = EarlyStopping(monitor = "accuracy",
                   min_delta = 0.01,
                   verbose = 1)

call_back = [mc,es]

# Model fitting

In [64]:
# Fitting the model 
model_new = model.fit(traindata, steps_per_epoch=60,epochs = 30,callbacks=call_back,validation_data=testdata)

Epoch 1/30
30/60 [==============>...............] - ETA: 6s - loss: 0.1065 - accuracy: 0.9901


Epoch 1: accuracy improved from 0.98731 to 0.99005, saving model to ./DN.h5
60/60 [==============================] - 12s 189ms/step - loss: 0.1065 - accuracy: 0.9901 - val_loss: 0.1061 - val_accuracy: 0.9869


# Exporting the model

In [65]:
# Exporting the model to json 
model_json = model.to_json()
with open("DN.json", "w") as json_file:
    json_file.write(model_json)


# Exporting the model weights

model.save_weights("DN")
print("Saved model to disk")




Saved model to disk


# Test The Model

In [67]:

#Testing the model 
predictions = ["Corpse Flower", 
               "Great Indian Bustard", 
               "Lady's slipper orchid", 
               "Pangolin", 
               "Spoon Billed Sandpiper", 
               "Seneca White Deer"
              ]


path = '/content/data/Testing/LS_Orchid_AUG/aug_download (1)_0_3466.jpg'
ime = tf.keras.utils.load_img(path,target_size=(224,224))

i = tf.keras.preprocessing.image.img_to_array(ime)
i = preprocess_input(i)
input = np.array([i])
pred = model.predict(input)
predictions[np.argmax(pred)]


1/1 [==============================] - 0s 25ms/step


"Lady's slipper orchid"